In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests

In [2]:
# URL of page to be scraped
url = 'https://www.walgreens.com/store/c/halloween-candy/ID=520931-tier3'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

o":{"ruleMessage":{"code":"WAG-R-VPD-0001","type":"RULE","prefix":"Buy 1, Get 1 50% OFF"},"regularPrice":"$1.29","regularPriceHtml":"<span class=\"product__price\"><sup>$</sup>1<sup>29</sup></span>","unitPrice":"$3.69","unitPriceSize":"oz","onSale":false},"averageRating":"0.0","subBrandName":"My Favorite","dsSkuId":"622255","pluCodeLinkText":"Mix & match products","fsa_cd":"0","inventoryText":"In stock at store","inventoryTextColor":"color__text-green-uc"}},{"productInfo":{"hairToolType":"","storeInv":"instock","fulfillerType":"DC","webExclusive":"0","wic":"686456","skuInvAvailMap":{"400622258":"1000"},"prodId":"300400865","skuId":"400622258","imageUrl":"//pics.drugstore.com/prodimg/622258/155.jpg","upc":"04142004537","productURL":"/store/c/ferrara-candy-nerds-grape,-strawberry--crunch-minis/ID=300400865-product","productSize":"11.6 oz ","newItem":"No","autoReorder":"0","unitPrice":"0.26","unitPriceSize":"oz","loyaltyEligible":"1","inactiveIngredients":"Dextrose,Sugar,Malic Acid,2% or 

In [6]:
# results are returned as an iterable list
results = soup.find_all('li', class_="result-row")

In [7]:
# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        # Identify and return price of listing
        price = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']

        # Print results only if title, price, and link are available
        if (title and price and link):
            print('-------------')
            print(title)
            print(price)
            print(link)
    except AttributeError as e:
        print(e)